### Notebook for analysis of cleansed data

### Milestone 7:

This query finds how many artists of each gender are represented in our data. The Artist table is JOINED with the Gender table and then GROUPED BY the gender type of each artist and the corresponding english word for that type. COUNT is used as the aggregate function to count how many artists belong to each gender type. ORDER BY is used to sort the most populous gender types from the less populous.

In [50]:
%%bigquery
select count(artist.gender) as Counts, gender.gender_type as Type from musicbrainz_modeled.Artist_Beam_DF as artist
join musicbrainz_modeled.Gender as gender on artist.gender = gender.gender_id
group by artist.gender, gender.gender_type
union all
select count(*), 'nulls' from musicbrainz_modeled.Artist_Beam_DF as a where a.gender is null
order by counts

,Counts,Type
0,380,Not applicable
1,746,Other
2,140187,Female
3,506998,Male
4,956060,nulls


This query finds all the places in each area that have long names. The Area table is JOINED with the Place table and GROUPED BY the place and area names that share the same area_id. The groups that HAVE place names with more than 15 letters are retained. The results are ORDERED BY the area_id so that continental and country level data is shown before more localalized smaller areas.

In [20]:
%%bigquery
select place.place_name as Places, area.area_name as Areas, area.area_id from musicbrainz_modeled.Place_Beam_DF as place
join musicbrainz_modeled.Area_Beam_DF as area on area.area_id = place.area_id
group by place.place_name, area.area_name, area.area_id
having length(place.place_name) > 15
order by area_id

,Places,Areas,area_id
0,Core Music Factory,Andorra,5
1,Rothera Research Station,Antarctica,8
2,Colombia Records,Argentina,10
3,Digisound Mastering,Argentina,10
4,The Garden Mastering,Argentina,10
...,...,...,...
19386,Ye Olde Orchard Pub & Grill,Monkland Village,118964
19387,Segal Centre for Performing Arts,Snowdon,118965
19388,Montreal Sound Studio,Snowdon,118965
19389,Playhouse Studio,Snowdon,118965


This query associates events with event types. The Event table is JOINED with the Event_Type table and is then GROUPED BY the event name and type.

In [31]:
%%bigquery
select event.event_name, et.type from musicbrainz_modeled.Event_Beam_DF as event
join musicbrainz_modeled.Event_Type as et on event.event_type = et.event_type_id
group by event.event_name, et.type

,event_name,type
0,Nicolas Fortin Scholarship,Masterclass/Clinic
1,Yamaha Tone Made Easy,Masterclass/Clinic
2,Aids Walk+Ride Charity,Masterclass/Clinic
3,Donegal Fiddlers’ Summer School 1995,Masterclass/Clinic
4,Saleem Ashkar Piano Masterclass,Masterclass/Clinic
...,...,...
30968,コミックマーケット91,Convention/Expo
30969,コミックマーケット93,Convention/Expo
30970,コミックマーケット95,Convention/Expo
30971,コミックマーケット97,Convention/Expo


This query identifies which languages have been used in over 10,000 releases. The Language table is JOINED with the Release table so that the language names can be paired with the language id of each release. The table is GROUPED BY the language used in the Release where each language grouped must HAVE 10,000 releases in that language. Finally, the resuls are ORDERED so that the langauge used for the most releases is shown first.

In [34]:
%%bigquery
select Lang.language_name as ReleaseLanguage, count(Rel.rel_id) as NumRels
from musicbrainz_modeled.Release_Beam_DF as Rel
    join musicbrainz_modeled.Language_Beam_DF as Lang
        on Rel.language = Lang.language_id
group by Lang.language_name
    having NumRels > 10000
order by NumRels desc

,ReleaseLanguage,NumRels
0,English,1633739
1,Japanese,115896
2,[Multiple languages],79911
3,German,71217
4,Spanish,67811
5,French,64727
6,Italian,22383
7,Portuguese,21935
8,Russian,18361
9,Finnish,18128


This query identifies the top ten cities from where music artists start their careers. The Area table is JOINED with the Artist table so that the artists can be GROUPED BY the cities (areas with area_type = 3) where they started their careers. Then the results are ORDERED BY which cities produced the most artists.

In [36]:
%%bigquery
select a.area_name as StartCity, count(*) as NumArtists
from musicbrainz_modeled.Artist_Beam_DF as artist
    join musicbrainz_modeled.Area_Beam_DF as a
        on artist.begin_area_id = a.area_id
    where a.area_type = 3
group by a.area_name
order by NumArtists desc
limit 10

,StartCity,NumArtists
0,London,5181
1,New York,3456
2,Los Angeles,3149
3,Paris,2383
4,Chicago,2330
5,Berlin,1904
6,Wien,1667
7,Toronto,1326
8,Philadelphia,1294
9,Buenos Aires,1213


This query uses a nested query to identify the extinct label (a label that is no longer used in the current day) with the longest period of activity measured in years. The Label table is filtered so that the MAX duration of a label's activities that was found in the nested query is matched with the label(s) name and start and end date.

In [39]:
%%bigquery
select l1.label_name, l1.begin_year, l1.end_year, (l1.end_year - l1.begin_year) as duration
from musicbrainz_modeled.Label_Beam_DF as l1
where (l1.end_year - l1.begin_year) = (
  select max(l2.end_year - l2.begin_year) 
  from musicbrainz_modeled.Label_Beam_DF as l2
  )

,label_name,begin_year,end_year,duration
0,Cotta'sche Buchhandlung,1659,1977,318


### Make Views for Data Studio Report

A report will be made for the top ten cities to have been the starting places of Artists' careers. Thus, a view is made below that can be imported into data studio.

In [2]:
%%bigquery
create or replace view musicbrainz_modeled.v_ten_artist_cities as (
select a.area_name as StartCity, count(*) as NumArtists
from `earnest-keep-266820.musicbrainz_modeled.Artist_Beam_DF` as artist
    join `earnest-keep-266820.musicbrainz_modeled.Area_Beam_DF` as a
        on artist.begin_area_id = a.area_id
    where a.area_type = 3
group by a.area_name
order by NumArtists desc
limit 10
)

""


Another view is made so that information describing the genders of different artists can be included in the Data Studio report

In [7]:
%%bigquery
create or replace view musicbrainz_modeled.v_artist_genders as (
select count(artist.gender) as Counts, gender.gender_type as Type 
from `earnest-keep-266820.musicbrainz_modeled.Artist_Beam_DF` as artist
join `earnest-keep-266820.musicbrainz_modeled.Gender` as gender on artist.gender = gender.gender_id
group by artist.gender, gender.gender_type
union all
select count(*), 'nulls' from `earnest-keep-266820.musicbrainz_modeled.Artist_Beam_DF` as a where a.gender is null
order by counts
)

""


### Milestone 8:

### Queries with Subqueries

This query finds all female Canadian artists ordered by their artist ids.

In [17]:
%%bigquery
select a.artist_id, a.artist_name, g.gender_id, g.gender_type from musicbrainz_modeled.Artist_Beam_DF a
left join musicbrainz_modeled.Gender g on a.gender = g.gender_id
where a.area_id in
    (select ar.area_id from musicbrainz_modeled.Area_Beam_DF ar
     where ar.area_name = "Canada")
and g.gender_id = 2
order by a.artist_id

,artist_id,artist_name,gender_id,gender_type
0,40,Alanis Morissette,2,Female
1,139,Céline Dion,2,Female
2,324,k.d. lang,2,Female
3,973,Shania Twain,2,Female
4,977,Loreena McKennitt,2,Female
...,...,...,...,...
2027,1931235,Diamond Tooth Molly,2,Female
2028,1931464,Elaine Keillor,2,Female
2029,1932342,Jessica Lalonde,2,Female
2030,1932465,Mary Jane Puiu,2,Female


Find all the songs by Canadian artists that are longer than twice average length (495577.398689).

In [36]:
%%bigquery
select a.artist_name, r.length from musicbrainz_modeled.Artist_Beam_DF a
join musicbrainz_modeled.Recording_Beam_DF r on r.artist_id = a.artist_id
where r.length >
    (select AVG(r.length)*2 from musicbrainz_modeled.Recording_Beam_DF r)
and a.area_id = 38
order by a.artist_name

,artist_name,length
0,(((stereofect))),642000
1,(((stereofect))),540000
2,(((stereofect))),498000
3,(((stereofect))),682000
4,(((stereofect))),1594000
...,...,...
10000,劉芳,528653
10001,劉芳,675000
10002,劉芳,758000
10003,劉芳,541000


The query below identifies the top ten artists that created the most recordings that are stored in the musicbrainz database. A subquery uses count() and GROUP BY to count how many recordings are associated with each artist_id. The results are used as a virtual table that is joined in an outer query with the Artists table, where information such as the artists' names can be connected with the results of the inner query. The artist_ids of 97546 and 105725 are omitted from the top ten list because they respectively correspond to the entries for [unkown] artist and [no artist].

In [5]:
%%bigquery
select AB.artist_name, ArtRecs.num_recs
from musicbrainz_modeled.Artist_Beam_DF as AB
join 

  (select RB.artist_id as rec_art_id, count(RB.artist_id) as num_recs
  from musicbrainz_modeled.Recording_Beam_DF as RB
  group by RB.artist_id) as ArtRecs
    
on AB.artist_id = ArtRecs.rec_art_id
where artist_id != 97546 and artist_id != 105725
order by ArtRecs.num_recs desc
limit 10

,artist_name,num_recs
0,Elvis Presley,24355
1,Bruce Springsteen,21241
2,The Beatles,21047
3,Martin Mook Knudsen,20293
4,Pearl Jam,20037
5,U2,19819
6,Grateful Dead,18510
7,The Rolling Stones,16702
8,Phish,15448
9,Bob Dylan,15420


The following query is very similar to the query above in that it produces a top ten of artists. However, here the list is of artists who produced the most albums. The where clause in the subquery filters release groups, collections of recordings released to the public, for those that were albums (type 1) rather than release groups such as EPs, singles, or radio broadcasts. Note that artist_id = 1 is excluded because this corresponds to the record for 'Various Artists', in which case albums produced by artist_id = 1 are actually ablums containing compilations of works made from various artists. Note that several of the artists that produced the most recordings (see the query above) are also in the list of artists with the most released albums (see the query below). Interestingly, the top three artists to produce the most albums are not even in the top ten of artists to produce the most recordings.

In [9]:
%%bigquery
select AB.artist_name, ArtRGs.num_albums
from musicbrainz_modeled.Artist_Beam_DF as AB
join 

  (select RGB.artist_id as rel_art_id, count(RGB.artist_id) as num_albums
  from musicbrainz_modeled.Release_Group_Beam_DF as RGB
  where RGB.rel_gr_type = 1
  group by RGB.artist_id) as ArtRGs
    
on AB.artist_id = ArtRGs.rel_art_id
where artist_id != 97546 and artist_id != 105725 and artist_id != 1
order by ArtRGs.num_albums desc
limit 10

,artist_name,num_albums
0,Sun Blossoms,17267
1,Kamikaze Girls,2107
2,Mercy Valdez,1771
3,U2,940
4,Bruce Springsteen,871
5,Grateful Dead,748
6,Pearl Jam,709
7,Phish,703
8,Elvis Presley,697
9,The Rolling Stones,665


### Make Views for Datastudio

A view is made for the top ten artists to produce the most recordings.

In [12]:
%%bigquery
create or replace view musicbrainz_modeled.v_most_recording_artists as 
(
select AB.artist_name, ArtRecs.num_recs
from `earnest-keep-266820.musicbrainz_modeled.Artist_Beam_DF` as AB
join 

  (select RB.artist_id as rec_art_id, count(RB.artist_id) as num_recs
  from `earnest-keep-266820.musicbrainz_modeled.Recording_Beam_DF` as RB
  group by RB.artist_id) as ArtRecs
    
on AB.artist_id = ArtRecs.rec_art_id
where artist_id != 97546 and artist_id != 105725
order by ArtRecs.num_recs desc
limit 10
)

""
